In [ ]:
import os
import zipfile
import re

Mounted at /content/drive


In [ ]:
folder1 = "TFG/DeepXplore/ImageNet/Resultats_DeepXplore_imagenet"
folder2 = "TFG/DLFuzz/ImageNet/Resultats_imagenet"

def process_folder(folder):
    zip_files = [f for f in os.listdir(folder) if f.endswith('.zip')]
    txt_files = [f for f in os.listdir(folder) if f.endswith('.txt')]
    txt_paths = []

    for zip_file in zip_files:
        zip_path = os.path.join(folder, zip_file)
        extract_path = os.path.join('/content', zip_file.replace(".zip", ""))

        os.makedirs(extract_path, exist_ok=True)

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extracted {zip_file} to {extract_path}")

    for txt_file in txt_files:
        txt_path = os.path.join(folder, txt_file)
        print(f"Found TXT file: {txt_path}")
        txt_paths.append(txt_path)

    return txt_paths


txt_deepxplore = process_folder(folder1)
txt_dlfuzz = process_folder(folder2)


In [ ]:
with zipfile.ZipFile("TFG/peikexin9-deepxplore-master.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
def get_img_paths(data):
    imgs_paths = set()
    lines = data.strip().split("\n")
    for line in lines:
        if line.startswith("Image path:"):
            img_path = line.split(":")[1].strip()
            imgs_paths.add(img_path)
    return imgs_paths

print("Text files found:", txt_deepxplore)

unique_imgs_paths = set()

for txt_file in txt_deepxplore:
    with open(txt_file, "r") as f:
        info = f.read()
    unique_imgs_paths.update(get_img_paths(info))


unique_imgs_paths_list = list(unique_imgs_paths)

print(unique_imgs_paths_list)


Text files found: ['/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_blackout2.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_blackout1.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_blackout0.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_occl2.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_occl1.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_occl0.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_light0.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_light2.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_light1.txt']
['./seeds/ILSVRC2012_test_00001568.JPEG', './seeds/ILSVRC2012_test_00000854.JPEG', './seeds/ILSVRC2012_test_00001120.JPEG', './seeds/ILSVRC2012_test_00001509.JPEG', './seeds/ILSVRC2012_test_00001550.JPEG', './seeds/ILSVRC2012_test_000002

In [ ]:
imgs_paths = []
def get_img_paths(data):
    lines = data.strip().split("\n")
    for i, line in enumerate(lines):
        if line.startswith("Image path:"):
            img_path = line.split(":")[1].strip()
            imgs_paths.append(img_path)
    print(imgs_paths)
    return imgs_paths

print(txt_deepxplore)
with open(txt_deepxplore[0], "r") as f:
    info = f.read()
deepxplore_paths = get_img_paths(info)

['/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_blackout2.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_blackout1.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_blackout0.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_occl2.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_occl1.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_occl0.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_light0.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_light2.txt', '/content/drive/MyDrive/GIA/TFG/DeepXplore/Resultats/output_ImageNet_light1.txt']
['./seeds/ILSVRC2012_test_00001208.JPEG', './seeds/ILSVRC2012_test_00001793.JPEG', './seeds/ILSVRC2012_test_00001629.JPEG', './seeds/ILSVRC2012_test_00001604.JPEG', './seeds/ILSVRC2012_test_00001158.JPEG', './seeds/ILSVRC2012_test_00000670.JPEG', './seeds

In [ ]:
with open(txt_dlfuzz[0], "r") as f:
    info = f.read()
dlfuzz_paths = get_img_paths(info)

In [ ]:
print(len(dlfuzz_paths))
len(deepxplore_paths)

343


100

In [ ]:
def get_img_paths_1(file, imgs_dict):
    with open(file, "r") as f:
        data = f.read()
    lines = data.strip().split("\n")

    img_path = None
    txt_name = os.path.splitext(os.path.basename(file))[0][len("output_"):]

    for i, line in enumerate(lines):
        if line.startswith("Image path:"):
            img_path = line.split(":")[1].strip()
            if img_path not in imgs_dict:
                imgs_dict[img_path] = []

        elif img_path and line.startswith("Models predictions before:"):
            for j in range(i + 1, len(lines)):
                if lines[j].startswith("./generated_inputs"):
                    original_filename = os.path.basename(lines[j].strip())
                    modified_path = f"./generated_inputs_{txt_name}/{original_filename}"
                    imgs_dict[img_path].append(modified_path)
                    break

    return imgs_dict

deepxplore_dict = {}
for txt_file in txt_deepxplore:
    deepxplore_dict = get_img_paths_1(txt_file, deepxplore_dict)

print(deepxplore_dict)


{'./seeds/ILSVRC2012_test_00001208.JPEG': ['./generated_inputs_ImageNet_occl2/occl_lifeboat_lifeboat_castle_0.png', './generated_inputs_ImageNet_occl1/occl_lifeboat_container_ship_lifeboat_0.png', './generated_inputs_ImageNet_occl0/occl_dock_lifeboat_lifeboat_0.png'], './seeds/ILSVRC2012_test_00001793.JPEG': [], './seeds/ILSVRC2012_test_00001629.JPEG': ['./generated_inputs_ImageNet_blackout2/blackout_patio_patio_boathouse_2.png'], './seeds/ILSVRC2012_test_00001604.JPEG': [], './seeds/ILSVRC2012_test_00001158.JPEG': [], './seeds/ILSVRC2012_test_00000670.JPEG': [], './seeds/ILSVRC2012_test_00000434.JPEG': [], './seeds/ILSVRC2012_test_00000388.JPEG': [], './seeds/ILSVRC2012_test_00001710.JPEG': [], './seeds/ILSVRC2012_test_00000490.JPEG': [], './seeds/ILSVRC2012_test_00000303.JPEG': ['./generated_inputs_ImageNet_blackout2/blackout_dining_table_desk_dining_table_10.png'], './seeds/ILSVRC2012_test_00000164.JPEG': [], './seeds/ILSVRC2012_test_00001370.JPEG': [], './seeds/ILSVRC2012_test_0000

In [ ]:
def remove_empty_list_keys(d):
    return {k: v for k, v in d.items() if v != []}

deepxplore_dict = remove_empty_list_keys(deepxplore_dict)

In [ ]:
def get_img_paths_2(file, imgs_dict):
    with open(file, "r") as f:
        data = f.read()
    lines = data.strip().split("\n")

    img_path = None
    generated_paths = []
    txt_name = os.path.splitext(os.path.basename(file))[0][len("output_"):]

    for line in lines:
        #print(line)
        if line.startswith("Image path:"):
            if img_path and generated_paths:
                new_img_path = img_path.replace('./seeds_per_DLFuzz/', './seeds/')
                imgs_dict[new_img_path] = generated_paths

            img_path = line.split(":")[1].strip()
            generated_paths = []

        elif img_path and line.startswith("generated image:"):
            original_filename = os.path.basename(line.strip())
            modified_path = f"./generated_inputs_{txt_name}/{original_filename}"
            generated_paths.append(modified_path)

    if img_path and generated_paths:
        new_img_path = img_path.replace('./seeds_per_DLFuzz/', './seeds/')
        imgs_dict[new_img_path] = generated_paths

    return imgs_dict

dlfuzz_dict = {}
for txt_file in txt_dlfuzz:
    dlfuzz_dict = get_img_paths_2(txt_file, dlfuzz_dict)

print(dlfuzz_dict)


{'./seeds/ILSVRC2012_test_00000721.JPEG': ['./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273079798.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273080504.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273081228.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273081994.png'], './seeds/ILSVRC2012_test_00001883.JPEG': ['./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/Egyptian_cat-mink-309273084520.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/Egyptian_cat-mink-309273085289.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/Egyptian_cat-mink-309273086049.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/Egyptian_cat-mink-309273086815.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/book_jacket-Egyptian_cat-309273092782.png', './generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/paper_towel-Egyptian_cat-309273093550.png', './generated_input



---



In [ ]:
#Per a fitxers DeepXplore
def get_img_paths_with_predictions(file, imgs_dict):
    with open(file, "r") as f:
        data = f.read()
    lines = data.strip().split("\n")

    img_path = None
    txt_name = os.path.splitext(os.path.basename(file))[0][len("output_"):]  
    predictions_after = ""

    for i, line in enumerate(lines):
        if line.startswith("Image path:"):
            img_path = line.split(":")[1].strip()
            if img_path not in imgs_dict:
                imgs_dict[img_path] = []

        elif line.startswith("Models predictions after:"):
            predictions_after = line

        elif img_path and line.startswith("./generated_inputs"):
            original_filename = os.path.basename(line.strip())
            modified_path = f"./generated_inputs_{txt_name}/{original_filename}"

            dir_name = os.path.basename(os.path.dirname(modified_path))
            result_num = int(dir_name.rsplit("_", 1)[-1][-1])+1

            prediction = "Unknown"
            if predictions_after:
                pattern = re.compile(rf"Model {result_num} -> ([^,]+ \(Conf: ([^\)]+)\)), \(Original predicion conf: ([^\)]+)\)")
                match = pattern.search(predictions_after)
                if match:
                    act_prediction = match.group(1)
                    act_prediction = act_prediction.split(" (")
                    act_prediction = "Pred_Class: " + act_prediction[0] + ", " + act_prediction[1].replace(")", "")
                    original_conf = match.group(3)
                    original_conf = "OGClass_conf: " + original_conf
                    prediction = act_prediction + ", " + original_conf

            if prediction !="Unknown":
              imgs_dict[img_path].append((modified_path, prediction))

    return imgs_dict

deepxplore_dict = {}
for txt_file in txt_deepxplore:
    deepxplore_dict = get_img_paths_with_predictions(txt_file, deepxplore_dict)

print(deepxplore_dict)

{'./seeds/ILSVRC2012_test_00001208.JPEG': [('./generated_inputs_ImageNet_occl2/occl_lifeboat_lifeboat_castle_0.png', 'Pred_Class: castle, Conf: 0.2466, OGClass_conf: 0.1071'), ('./generated_inputs_ImageNet_occl1/occl_lifeboat_container_ship_lifeboat_0.png', 'Pred_Class: container_ship, Conf: 0.6476, OGClass_conf: 0.1150'), ('./generated_inputs_ImageNet_occl0/occl_dock_lifeboat_lifeboat_0.png', 'Pred_Class: dock, Conf: 0.3555, OGClass_conf: 0.1804')], './seeds/ILSVRC2012_test_00001793.JPEG': [], './seeds/ILSVRC2012_test_00001629.JPEG': [('./generated_inputs_ImageNet_blackout2/blackout_patio_patio_boathouse_2.png', 'Pred_Class: boathouse, Conf: 0.4307, OGClass_conf: 0.4191')], './seeds/ILSVRC2012_test_00001604.JPEG': [], './seeds/ILSVRC2012_test_00001158.JPEG': [], './seeds/ILSVRC2012_test_00000670.JPEG': [], './seeds/ILSVRC2012_test_00000434.JPEG': [], './seeds/ILSVRC2012_test_00000388.JPEG': [('./generated_inputs_ImageNet_blackout2/already_differ_fountain_fountain_soccer_ball_7.png', '

In [ ]:
#Per als fitxers DLFuzz
def get_img_paths_with_predictions_2(file, imgs_dict):
    with open(file, "r") as f:
        data = f.read()
    lines = data.strip().split("\n")

    img_path = None
    generated_paths = []
    txt_name = os.path.splitext(os.path.basename(file))[0][len("output_"):]
    prediction_after = "Unknown"

    for line in lines:
        if line.startswith("Image path:"):
            if img_path and generated_paths:
                new_img_path = img_path.replace('./seeds_per_DLFuzz/', './seeds/')
                imgs_dict[new_img_path] = [(path, prediction_after) for path in generated_paths]

            img_path = line.split(":")[1].strip()
            generated_paths = []
            prediction_after = "Unknown"

        elif line.startswith("Models predictions after:"):
            match = re.search(r"Models predictions after: (.+) \(([0-9\.]+)\)", line)
            if match:
                prediction_after = "Pred_Class: " + match.group(1) + ", Conf: " + match.group(2)

        elif img_path and line.startswith("generated image:"):
            original_filename = os.path.basename(line.strip())
            modified_path = f"./generated_inputs_{txt_name}/{original_filename}"
            generated_paths.append(modified_path)

    if img_path and generated_paths:
        new_img_path = img_path.replace('./seeds_per_DLFuzz/', './seeds/')
        imgs_dict[new_img_path] = [(path, prediction_after) for path in generated_paths]

    return imgs_dict

dlfuzz_dict = {}
for txt_file in txt_dlfuzz:
    dlfuzz_dict = get_img_paths_with_predictions_2(txt_file, dlfuzz_dict)

print(dlfuzz_dict)


{'./seeds/ILSVRC2012_test_00000721.JPEG': [('./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273079798.png', 'Pred_Class: vestment, Conf: 0.271183788776'), ('./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273080504.png', 'Pred_Class: vestment, Conf: 0.271183788776'), ('./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273081228.png', 'Pred_Class: vestment, Conf: 0.271183788776'), ('./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/vestment-kimono-309273081994.png', 'Pred_Class: vestment, Conf: 0.271183788776')], './seeds/ILSVRC2012_test_00001883.JPEG': [('./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/Egyptian_cat-mink-309273084520.png', 'Pred_Class: bath_towel, Conf: 0.975012421608'), ('./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/Egyptian_cat-mink-309273085289.png', 'Pred_Class: bath_towel, Conf: 0.975012421608'), ('./generated_inputs_DLFuzz_ImageNet_imgdeep_vgg16_0/Egyptian_cat-mink-309273086049.png', 'Pred_

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import json
import urllib.request


def plot_images_from_dict(imgs_dict_1, imgs_dict_2):
    common_keys = {key for key in imgs_dict_1.keys() & imgs_dict_2.keys() if imgs_dict_1[key] and imgs_dict_2[key]}
    url = "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json"
    response = urllib.request.urlopen(url)
    imagenet_labels = json.load(response)

    model = tf.keras.applications.EfficientNetB7(weights='imagenet')

    for img_path in common_keys:
        generated_paths_1 = imgs_dict_1[img_path]
        generated_paths_2 = imgs_dict_2[img_path]
        total_images = 1 + len(generated_paths_1) + len(generated_paths_2)

        plt.figure(figsize=(5 * total_images, 5))

        img = mpimg.imread(img_path)


        img = image.load_img(img_path, target_size=(600, 600))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
        predictions = model.predict(img_array, verbose=0)
        label = tf.keras.applications.efficientnet.decode_predictions(predictions, top=1)[0][0][1]


        plt.subplot(1, total_images, 1)
        plt.imshow(img)
        plt.title(f"Original Image: {os.path.basename(img_path)}\nClass: {label}")
        plt.axis('off')

        for i, (gen_path, pred_after) in enumerate(generated_paths_1):
            img_gen = mpimg.imread(gen_path)
            plt.subplot(1, total_images, i + 2)
            plt.imshow(img_gen)
            dir_name = os.path.basename(os.path.dirname(gen_path))
            result = dir_name.rsplit("_", 1)[-1]
            plt.title(f"DeepXplore - {result} \n{pred_after}")
            plt.axis('off')

        for i, (gen_path, pred_after) in enumerate(generated_paths_2):
            img_gen = mpimg.imread(gen_path)
            plt.subplot(1, total_images, len(generated_paths_1) + i + 2)
            plt.imshow(img_gen)
            plt.title(f"DLFuzz\n{pred_after}")
            plt.axis('off')

        plt.tight_layout()
        plt.show()

plot_images_from_dict(deepxplore_dict, dlfuzz_dict)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

def plot_images_from_dict(imgs_dict_1, imgs_dict_2):
    common_keys = set(imgs_dict_1.keys()) & set(imgs_dict_2.keys())

    for img_path in common_keys:
        generated_paths_1 = imgs_dict_1[img_path]
        generated_paths_2 = imgs_dict_2[img_path]
        total_images = 1 + len(generated_paths_1) + len(generated_paths_2)

        plt.figure(figsize=(5 * total_images, 5))

        img = mpimg.imread(img_path)
        plt.subplot(1, total_images, 1)
        plt.imshow(img)
        plt.title(f"Original Image: {os.path.basename(img_path)}")
        plt.axis('off')

        for i, gen_path in enumerate(generated_paths_1):
            img_gen = mpimg.imread(gen_path)
            plt.subplot(1, total_images, i + 2)
            plt.imshow(img_gen)
            #print(gen_path)
            dir_name = os.path.basename(os.path.dirname(gen_path))
            result = dir_name.rsplit("_", 1)[-1]
            plt.title(f"Generated Image DeepXplore - {result}")
            plt.axis('off')

        for i, gen_path in enumerate(generated_paths_2):
            img_gen = mpimg.imread(gen_path)
            plt.subplot(1, total_images, len(generated_paths_1) + i + 2)
            plt.imshow(img_gen)
            plt.title(f"Generated Image DLFuzz - {i+1}")
            plt.axis('off')

        plt.tight_layout()
        plt.show()

plot_images_from_dict(deepxplore_dict, dlfuzz_dict)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

def plot_images_for_single_key(imgs_dict_1, imgs_dict_2, target_key):
    if target_key in imgs_dict_1:
        generated_paths_1 = imgs_dict_1[target_key]
        generated_paths_2 = imgs_dict_2.get(target_key, [])

        total_images = 1 + len(generated_paths_1) + len(generated_paths_2)

        plt.figure(figsize=(5 * total_images, 5))

        img = mpimg.imread(target_key)
        plt.subplot(1, total_images, 1)
        plt.imshow(img)
        plt.title(f"Original Image: {os.path.basename(target_key)}")
        plt.axis('off')

        for i, gen_path in enumerate(generated_paths_1):
            img_gen = mpimg.imread(gen_path)
            plt.subplot(1, total_images, i + 2)
            plt.imshow(img_gen)
            plt.title(f"Generated Image 1-{i+1}")
            plt.axis('off')

        for i, gen_path in enumerate(generated_paths_2):
            img_gen = mpimg.imread(gen_path)
            plt.subplot(1, total_images, len(generated_paths_1) + i + 2)
            plt.imshow(img_gen)
            plt.title(f"Generated Image 2-{i+1}")
            plt.axis('off')

        plt.tight_layout()
        plt.show()

    else:
        print(f"The key {target_key} was not found in the first dictionary.")

target_key = './seeds/ILSVRC2012_test_00001901.JPEG'
plot_images_for_single_key(deepxplore_dict, dlfuzz_dict, target_key)
